# Onnx to Circom

 ## Imports

In [55]:
import onnx
import tensorflow as tf

## Parameters extraction

### Onnx model

In [56]:
useful_operations = [
    "Conv",
    "BatchNormalization",
    "MatMul",
    "Relu",
    "AveragePool",
    "MaxPool",
    "GlobalMaxPool",
    "GlobalAveragePool",
    "Softmax"
]

In [57]:
# Load the ONNX model
model = onnx.load('model.onnx')


In [58]:
type(model.graph.node[0])

onnx.onnx_ml_pb2.NodeProto

In [59]:
# Iterate over the nodes
num = 0
for node in model.graph.node:
    num+=1
    print(node.op_type)
    if node.op_type in useful_operations:
        print(num)
        


Shape
Cast
Gather
Concat
Cast
Reshape
MatMul
7
Reshape
Add
Relu
10
Transpose
BatchNormalization
12
Transpose
Shape
Gather
Cast
Gather
ReduceProd
Unsqueeze
Gather
Concat
Cast
ReduceProd
Unsqueeze
Concat
Cast
Reshape
MatMul
28
Reshape
Add
Relu
31
Reshape
MatMul
33
Add
Softmax
35


### Tensorflow model

In [63]:
model_tf = tf.keras.models.load_model('model.h5')

In [64]:
for layer in model_tf.layers:
    print(layer.name, layer.trainable)

dense_3 True
batch_normalization_1 True
dense_4 True
layer_6 True
dense_5 True


### Layer Traslation

#### Conv layer

In [9]:
layer = model_tf.layers[0]
node = model.graph.node[8]

##### Layer name

In [65]:
layer.name, node.op_type

('conv1d', 'Conv')

##### Input
The input shape is the same of the previous layer's output shape or the first input calculated above

In [48]:
# It is the shape of the input tensor
layer.input

<KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'conv1d_input')>

##### Weights (Shape and values)
To be given thanks to json file

In [49]:
layer.weights[0].shape

#ONNX: load from json to security reasons

TensorShape([3, 3, 5])

##### Bias (bias) (Shapes and values)
To be given thanks to json file

In [17]:
layer.weights[1].shape

#ONNX: load from json to security reasons

TensorShape([5])

In [18]:
layer.weights

[<tf.Variable 'conv1d/kernel:0' shape=(3, 3, 5) dtype=float32, numpy=
 array([[[ 0.10489794,  0.02674789,  0.22496808, -0.13555515,
           0.32511517],
         [ 0.01517793,  0.33708185, -0.19704996, -0.31072298,
          -0.02127604],
         [-0.1836963 , -0.28111267,  0.01842831,  0.05202194,
          -0.20682344]],
 
        [[-0.46488547,  0.39390826, -0.04661333,  0.12807512,
          -0.27943522],
         [-0.00076935,  0.28162053,  0.32568017,  0.35260952,
           0.3902165 ],
         [-0.49525642, -0.11718595, -0.41158077, -0.02413559,
           0.23505296]],
 
        [[ 0.33310878, -0.2687955 , -0.25753167, -0.06768823,
           0.18005817],
         [ 0.45210898,  0.11931844, -0.45146644,  0.15307127,
          -0.331387  ],
         [-0.2640147 , -0.2364319 , -0.49394742,  0.386068  ,
          -0.48843566]]], dtype=float32)>,
 <tf.Variable 'conv1d/bias:0' shape=(5,) dtype=float32, numpy=
 array([ 1.1342889e-06,  6.9391881e-06, -1.3836824e-06,  2.6138882e-

##### Layer output
To be calculated, in case of Conv layer:

- H_out = [(H_in - F_height + 2 * Padding) / S_height] + 1

- W_out = [(W_in - F_width + 2 * Padding) / S_width] + 1

- C_out = Number of filters in the convolutional layer

In [51]:
layer.output #input: 4, 10, 32

<KerasTensor: shape=(None, 32, 30, 5) dtype=float32 (created by layer 'conv1d')>

In [189]:
# (filter dim1, filter dim2, input channels, output channels)
layer.get_weights()[0].shape, layer.padding

Kernel shape:  [3, 3]


In [190]:
# filter dim1, filter dim2
for attribute in node.attribute:
    if attribute.name == 'kernel_shape':
        kernel_shape = attribute.ints[1]
        print('Kernel shape: ', [kernel_shape, kernel_shape])

Kernel shape:  [3, 3]


##### nRows

In [186]:
layer.input[0] # 4

<KerasTensor: shape=(4, 10, 32) dtype=float32 (created by layer 'tf.__operators__.getitem_18')>

##### nCol

In [146]:
layer.input[1] # 10

<KerasTensor: shape=(4, 10, 32) dtype=float32 (created by layer 'tf.__operators__.getitem_16')>

##### nChannels & nFilters

In [166]:
# The number of channels is egual to the number of filters, 
# and it's egual to the number of dimensione 3 of wheits array
layer.filters, layer.weights[0].shape[3]

(5, 5)

In [528]:
input_shape

[32, 32, 3]

In [529]:
input_shape[-1]

[32, 32]

In [520]:
for node in model.graph.node:
    if node.op_type == 'Conv':  # We found a convolutional layer
        # Get the name of the weights tensor for this layer
        weights_name = node.input[1]  # The weights tensor is usually the second input
        # Now we look for this tensor in the initializers
        for initializer in model.graph.initializer:
            if initializer.name == weights_name:
                # We found the weights tensor. Its shape gives us the number of filters
                weights = numpy_helper.to_array(initializer)
                print('Layer (node) name: ', node.name)
                print('Number of filters: ', weights.shape[0])
                break


##### KernelSize

In [171]:
layer.kernel_size

(3, 3)

In [179]:
for attribute in node.attribute:
    if attribute.name == 'kernel_shape':
        kernel_shape = attribute.ints[1]
        print('Kernel shape: ', [kernel_shape, kernel_shape])

Kernel shape:  [3, 3]


##### Strides

In [182]:
for attribute in node.attribute:
    if attribute.name == 'strides':
        stride_shape = attribute.ints
        print('Strides shape: ', stride_shape)

Strides shape:  [1, 1]


#### Dense layer

In [532]:
layer = model_tf.layers[0]
node_mat = model.graph.node[6]
node_relu = model.graph.node[9]

In [611]:
node  = model.graph.node[6]
output_name = str(node.output[0])
output_names = output_name.split('/')
output_name = '/'.join(output_names[:2])
output_name

'sequential_1/dense_3'

In [599]:
for value_info in model.graph.initializer:
    if output_name in value_info.name:
        last = value_info
        print(value_info)
num_neurons = last.dims
print(f"The number of neurons in the node is {num_neurons}")

    

dims: 1
data_type: 6
name: "sequential_1/dense_3/Tensordot/axes:0"
raw_data: "\003\000\000\000"

dims: 2
data_type: 7
name: "sequential_1/dense_3/Tensordot/Reshape_shape__92"
raw_data: "\377\377\377\377\377\377\377\377\003\000\000\000\000\000\000\000"

dims: 3
dims: 8
data_type: 1
name: "sequential_1/dense_3/Tensordot/ReadVariableOp:0"
raw_data: "\322?q\276A\311\033\276\343:\262>c\322\223\276\267/\267\276\265\366\243>\036\254B\276TE$?u\335\335>!\277\317\276V\320\216\276\351\267\335>|\326\236\276F[\335>D  \277\214\022!\277q\033\333>\346\254\312\276\370&8\277\320\025.?~2\037\276A\335\n\277C\327\200\276\236\253e\276"

dims: 1
data_type: 6
name: "sequential_1/dense_3/Tensordot/Const_2:0"
raw_data: "\010\000\000\000"

dims: 8
data_type: 1
name: "sequential_1/dense_3/BiasAdd/ReadVariableOp:0"
raw_data: "\272_i\274\000\000\000\000\251k{\273n~\244;\000\000\000\000\307\360\021\274\000\000\000\000\212|J\273"

dims: 8
data_type: 1
name: "sequential_1/dense_3/BiasAdd/ReadVariableOp:0"
raw_data: "\

In [602]:
input_shape = (32, 32, 3)
input_shape[:-1] + (6,)

(32, 32, 6)

### Circuit Conv Creation

In [1]:
from keras2circom.keras2circom.circom import Component, Circuit, Signal, Template
import typing


In [2]:
import os
from os import path
import re

def dir_parse(dir_path, skips=[]):
    '''parse circom files in a directory'''
    names = os.listdir(dir_path)
    for name in names:
        if name in skips:
            continue

        fpath = path.join(dir_path, name)
        if os.path.isdir(fpath):
            dir_parse(fpath)
        elif os.path.isfile(fpath):
            if fpath.endswith('.circom'):
                templates = file_parse(fpath)
    print(templates)
    return templates

def file_parse(fpath):
    '''parse circom file and register templates'''
    with open(fpath, 'r') as f:
        lines = f.read().split('\n')

    lines = [l for l in lines if not l.strip().startswith('//')]
    lines = ' '.join(lines)

    lines = re.sub('/\*.*?\*/', 'IGN', lines)

    funcs = re.findall('template (\w+) ?\((.*?)\) ?\{(.*?)\}', lines)
    for func in funcs:
        op_name = func[0].strip()
        args = func[1].split(',')
        main = func[2].strip()
        assert op_name not in templates, \
            'duplicated template: {} in {} vs. {}'.format(
                    op_name, templates[op_name].fpath, fpath)

        signals = re.findall('signal (\w+) (\w+)(.*?);', main)
        infos = [[] for i in range(4)]
        for sig in signals:
            sig_types = ['input', 'output']
            assert sig[0] in sig_types, sig[1] + ' | ' + main
            idx = sig_types.index(sig[0])
            infos[idx*2+0].append(sig[1])

            sig_dim = sig[2].count('[')
            infos[idx*2+1].append(sig_dim)
        templates[op_name] = Template(
                op_name, fpath,
                [a.strip() for a in args],
                *infos)
    return templates

In [3]:
templates = {}
templates = dir_parse('keras2circom/node_modules/circomlib-ml/circuits/', skips=['util.circom', 'circomlib-matrix', 'circomlib', 'crypto'])

{'ArgMax': Template(op_name='ArgMax', fpath='keras2circom/node_modules/circomlib-ml/circuits/ArgMax.circom', args=['n'], input_names=['in'], input_dims=[1], output_names=['out'], output_dims=[0]), 'AveragePooling2D': Template(op_name='AveragePooling2D', fpath='keras2circom/node_modules/circomlib-ml/circuits/AveragePooling2D.circom', args=['nRows', 'nCols', 'nChannels', 'poolSize', 'strides', 'scaledInvPoolSize'], input_names=['in'], input_dims=[3], output_names=['out'], output_dims=[3]), 'BatchNormalization2D': Template(op_name='BatchNormalization2D', fpath='keras2circom/node_modules/circomlib-ml/circuits/BatchNormalization2D.circom', args=['nRows', 'nCols', 'nChannels'], input_names=['in', 'a', 'b'], input_dims=[3, 1, 1], output_names=['out'], output_dims=[3]), 'Conv1D': Template(op_name='Conv1D', fpath='keras2circom/node_modules/circomlib-ml/circuits/Conv1D.circom', args=['nInputs', 'nChannels', 'nFilters', 'kernelSize', 'strides'], input_names=['in', 'weights', 'bias'], input_dims=[

In [4]:
import numpy as np
array = (np.random.random(1458) * 255).astype(np.uint8)

In [5]:
print("[", end="")
for i in array.flatten():
    print(str(i)+ ",",  end=" ")
print("]")

[185, 70, 22, 30, 50, 165, 247, 200, 174, 242, 163, 155, 249, 148, 116, 146, 239, 1, 185, 76, 200, 190, 131, 122, 112, 218, 120, 165, 150, 116, 249, 55, 203, 100, 235, 4, 251, 56, 114, 109, 108, 110, 175, 186, 74, 220, 63, 69, 160, 163, 160, 189, 126, 210, 76, 71, 124, 190, 12, 63, 174, 66, 133, 74, 177, 168, 42, 83, 243, 27, 110, 201, 39, 30, 186, 119, 88, 96, 85, 118, 208, 64, 70, 227, 44, 138, 222, 142, 199, 246, 204, 118, 180, 65, 225, 226, 147, 30, 189, 141, 99, 129, 75, 132, 91, 203, 107, 35, 72, 241, 50, 107, 195, 116, 118, 227, 127, 119, 29, 75, 196, 227, 163, 44, 106, 88, 148, 72, 85, 38, 108, 33, 192, 122, 139, 24, 249, 136, 117, 179, 193, 115, 42, 13, 42, 132, 78, 127, 61, 77, 231, 120, 145, 7, 113, 79, 64, 40, 26, 134, 221, 197, 109, 220, 60, 222, 85, 87, 41, 144, 248, 4, 172, 188, 125, 135, 0, 75, 252, 175, 39, 194, 40, 155, 87, 52, 195, 40, 30, 48, 232, 104, 60, 62, 48, 235, 108, 111, 89, 37, 94, 177, 7, 39, 223, 30, 224, 189, 88, 223, 162, 177, 139, 239, 142, 189, 189, 2

In [18]:
layer.input

<KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'dense_3_input')>

### Translation

In [509]:
circuit = Circuit()

In [519]:
input_name = model.graph.input[0].name
input_shape = [dim.dim_value for dim in model.graph.input[0].type.tensor_type.shape.dim][1:]
type(input_shape)

list

In [510]:
layer_1 = model_tf.layers[0]

In [507]:
layer_1.input[0]

<KerasTensor: shape=(32, 32, 3) dtype=float32 (created by layer 'tf.__operators__.getitem_176')>

In [608]:
node

input: "sequential_1/dense_3/Tensordot/Reshape:0"
input: "sequential_1/dense_3/Tensordot/ReadVariableOp:0"
output: "sequential_1/dense_3/Tensordot/MatMul:0"
name: "sequential_1/dense_3/Tensordot/MatMul"
op_type: "MatMul"

In [610]:
node.output[0]

'sequential_1/dense_3/Tensordot/MatMul:0'

In [511]:
dense_1 = Component(layer_1.name, templates['Dense'], [
    Signal('in', (32, 32, 3)),
    Signal('weights', layer_1.weights[0].shape, array),
    Signal('bias', layer_1.weights[1].shape, layer_1.weights[1]),
    ],[Signal('out', (32, 32, 8))],{
    'nInputs': 32 ,
    'nOutputs': 32,
    })

circuit.add_component(dense_1)


In [512]:
activation_1 = Component(layer_1.name+'_re_lu', templates['ReLU'], 
    [Signal('in', (32, 32, 8))], 
    [Signal('out', (32, 32, 8))])

circuit.add_component(activation_1)

In [486]:
layer_2 = model_tf.layers[1]

In [487]:
gamma = layer_2.weights[0]
beta = layer_2.weights[1]
moving_mean = layer_2.weights[2]
moving_var = layer_2.weights[3]
epsilon = layer_2.epsilon

a = gamma/(moving_var+epsilon)**.5
b = beta-gamma*moving_mean/(moving_var+epsilon)**.5

batch_1 = Component(layer_2.name, templates['BatchNormalization2D'], [
        Signal('in', (32,32,8)),
        Signal('a', a.shape, a),
        Signal('b', b.shape, b),
        ],[Signal('out',(32, 32, 8))],{
        'nRows': 32,
        'nCols': 32,
        'nChannels': 3,
        })

circuit.add_component(batch_1)

In [488]:
layer_3 = model_tf.layers[2]

In [489]:
dense_2 = Component(layer_3.name, templates['Dense'], [
    Signal('in', (32, 32, 8)),
    Signal('weights', layer_3.weights[0].shape, layer_3.weights[0]),
    Signal('bias', layer_3.weights[1].shape, layer_3.weights[1]),
    ],[Signal('out', (32, 32, 12))],{
    'nInputs': layer_3.input[0],
    'nOutputs': layer_3.output[0],
    })

circuit.add_component(dense_2)

In [490]:
activation_2 = Component(layer_3.name+'_re_lu', templates['ReLU'], 
    [Signal('in', (32, 32, 12))], 
    [Signal('out', (32, 32, 12))])

circuit.add_component(activation_2)

In [491]:
layer_4 = model_tf.layers[3]

In [492]:
flatten = Component(layer_4.name, templates['Flatten2D'], [
        Signal('in', (32, 32, 12)),
        ],[Signal('out', (12288,))],{
        'nRows': 32,
        'nCols': 32,
        'nChannels': 12,
        })

circuit.add_component(flatten)

In [493]:
layer_5 = model_tf.layers[4]

In [494]:
layer_5.input

<KerasTensor: shape=(None, 12288) dtype=float32 (created by layer 'layer_6')>

In [495]:
dense_3 = Component(layer_5.name, templates['Dense'], [
    Signal('in', (12288,)),
    Signal('weights', layer_5.weights[0].shape, layer_5.weights[0]),
    Signal('bias', layer_5.weights[1].shape, layer_5.weights[1]),
    ],[Signal('out', (4,))],{
    'nInputs': 12288,
    'nOutputs': 4,
    })

circuit.add_component(dense_3)

In [496]:
activation_3 = Component(layer_5.name+'_softmax', templates['ArgMax'],
    [Signal('in', (4,))],
    [Signal('out', (1,))], {'n': 4})

circuit.add_component(activation_3)

In [513]:
with open('circuit.json', 'w') as f:
    f.write(circuit.to_json())
    
with open('circuit.circom', 'w') as f:
    f.write(circuit.to_circom())